In [1]:
import pandas as pd
from collections import Counter

data = pd.read_csv("/content/updated(2)_dataset.csv")

#label
data["target"] = data["Track"] + "_" + data["Difficulty Level"]
data["text"] = (
    "Track: " + data["Track"] + ". " +
    "Difficulty Level: " + data["Difficulty Level"] + ". " +
    "Description: " + data["course_description"]
)

print("Distribution of Classes:")
print(Counter(data["target"]))

Distribution of Classes:
Counter({'Introduction to Computer Science_Beginner': 62, 'Data Science_Intermediate': 38, 'Operating Systems_Intermediate': 29, 'Data Science_Beginner': 24, 'Introduction to Computer Science_Intermediate': 22, 'Deep Learning_Intermediate': 19, 'Machine Learning / Data Science_Beginner': 17, 'Artificial Intelligence_Intermediate': 17, 'Data Science_Advanced': 16, 'Machine Learning_Beginner': 15, 'Programming  and Software Development_Beginner': 10, 'Machine Learning_Intermediate': 10, 'Artificial Intelligence_Advanced': 10, 'Web Development_Beginner': 9, 'Cybersecurity_Advanced': 8, 'Machine Learning_Advanced': 8, 'Operating Systems_Beginner': 8, 'Operating Systems_Advanced': 8, 'Cybersecurity_Beginner': 7, 'Programming  and Software Development_Intermediate': 7, 'Software Engineering_Intermediate': 7, 'Artificial Intelligence_Beginner': 7, 'Game Development_Intermediate': 7, 'Data Science / Machine Learning_Beginner': 6, 'Introduction to Computer Science_Advan

In [3]:
'''print("Missing values in 'Track':", data["Track"].isnull().sum())
print("Missing values in 'Difficulty Level':", data["Difficulty Level"].isnull().sum())
print("Missing values in 'course_description':", data["course_description"].isnull().sum())

data = data.dropna(subset=["Track", "Difficulty Level", "course_description"])
'''

Missing values in 'Track': 0
Missing values in 'Difficulty Level': 0
Missing values in 'course_description': 1


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Using Tf_iDF
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X = vectorizer.fit_transform(data["text"])

# training_test
X_train, X_test, y_train, y_test = train_test_split(X, data["target"], test_size=0.2, random_state=42)

In [6]:
 #!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np

In [8]:
# تحويل التسميات (target) إلى أرقام
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data["labels"] = label_encoder.fit_transform(data["target"])

print(data[["Course Name", "Track", "Difficulty Level", "target", "labels"]].head())

                                         Course Name  \
0       Retrieve Data using Single-Table SQL Queries   
1                               Hacking and Patching   
2  AWS Elastic Beanstalk: Build & Deploy a Node.j...   
3                  Cryptography and Hashing Overview   
4  Real-time OCR and Text Detection with Tensorfl...   

                                        Track Difficulty Level  \
0  Data Analysis /Database Management Systems         Beginner   
1                               Cybersecurity         Advanced   
2       Cloud Computing / Backend Development         Advanced   
3                               Cybersecurity     Intermediate   
4          Computer Vision / Machine Learning         Beginner   

                                              target  labels  
0  Data Analysis /Database Management Systems_Beg...      58  
1                             Cybersecurity_Advanced      44  
2     Cloud Computing / Backend Development_Advanced      19  
3             

In [9]:
from datasets import Dataset

dataset = Dataset.from_pandas(data)


In [10]:
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)

train_data = train_test_split["train"]
test_data = train_test_split["test"]

print("Training data size:", len(train_data))
print("Testing data size:", len(test_data))

Training data size: 462
Testing data size: 116


In [11]:
print("Columns in train_dataset:", train_data.column_names)


Columns in train_dataset: ['Course_ID', 'Course Name', 'Track', 'Difficulty Level', 'course_description', 'target', 'text', 'labels', '__index_level_0__']


In [12]:
from transformers import AutoTokenizer

# تحميل Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# وظيفة لتجهيز البيانات
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# تطبيق Tokenization على البيانات
tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_test = test_data.map(tokenize_function, batched=True)

# إضافة labels إلى البيانات
tokenized_train = tokenized_train.map(lambda examples: {"labels": examples["labels"]})
tokenized_test = tokenized_test.map(lambda examples: {"labels": examples["labels"]})

# عرض أول صف من البيانات المعدلة
print(tokenized_train[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/462 [00:00<?, ? examples/s]

Map:   0%|          | 0/116 [00:00<?, ? examples/s]

Map:   0%|          | 0/462 [00:00<?, ? examples/s]

Map:   0%|          | 0/116 [00:00<?, ? examples/s]

{'Course_ID': 564, 'Course Name': 'Change of Era: The Origins of Christian Culture through the Lens of Archaeology', 'Track': 'Deep Learning', 'Difficulty Level': 'Intermediate', 'course_description': 'highlight new jobrelevant skill youve gained supplement existing qualification hardearned industryspecific digital certificate plus one every course within experttrack', 'target': 'Deep Learning_Intermediate', 'text': 'Track: Deep Learning. Difficulty Level: Intermediate. Description: highlight new jobrelevant skill youve gained supplement existing qualification hardearned industryspecific digital certificate plus one every course within experttrack', 'labels': 88, '__index_level_0__': 562, 'input_ids': [101, 2650, 1024, 2784, 4083, 1012, 7669, 2504, 1024, 7783, 1012, 6412, 1024, 12944, 2047, 3105, 16570, 13331, 3372, 8066, 2017, 3726, 4227, 12448, 4493, 8263, 2524, 14644, 7228, 3068, 13102, 8586, 18513, 3617, 8196, 4606, 2028, 2296, 2607, 2306, 6739, 6494, 3600, 102, 0, 0, 0, 0, 0, 0, 0

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# تحميل النموذج
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_encoder.classes_)  # عدد الفئات
)

# إعداد معاملات التدريب
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir="./logs",  # Directory for storing logs
    logging_steps=10,  # Log training loss every 10 steps
    save_strategy="epoch",  # Save model at the end of each epoch
    save_total_limit=2,  # Keep only the last 2 checkpoints
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="eval_loss",  # Use validation loss to determine the best model
    greater_is_better=False,  # Lower validation loss is better
)

# إعداد Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

# بدء التدريب
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,4.629600,4.552560
2,4.100400,4.207198
3,3.731800,3.890340
4,3.387700,3.630803
5,3.074200,3.396601
6,2.668200,3.198992
7,2.739400,3.019742
8,2.466600,2.933727
9,2.213500,2.815709
10,2.051400,2.713491


TrainOutput(global_step=1160, training_loss=2.4391356813496556, metrics={'train_runtime': 1109.1184, 'train_samples_per_second': 8.331, 'train_steps_per_second': 1.046, 'total_flos': 2434398565294080.0, 'train_loss': 2.4391356813496556, 'epoch': 20.0})

In [18]:
# Update the number of epochs to 40 (20 already trained + 20 new)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,  # Total epochs (20 + 20)
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# إعداد Trainer (no changes needed here)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

# Resume training from the last checkpoint
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.879100,2.295291
2,1.373900,2.203045
3,1.389800,2.194987
4,1.573800,2.146958
5,1.252700,2.081278
6,1.006800,2.041945
7,1.277200,1.992548
8,1.058700,2.002919
9,0.980400,1.967185
10,0.912400,1.904485


TrainOutput(global_step=1160, training_loss=0.986374713223556, metrics={'train_runtime': 1559.4962, 'train_samples_per_second': 5.925, 'train_steps_per_second': 0.744, 'total_flos': 2434398565294080.0, 'train_loss': 0.986374713223556, 'epoch': 20.0})

In [42]:
# Update the number of epochs to 40 (20 already trained + 20 new)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,  # Total epochs (20 + 20)
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# إعداد Trainer (no changes needed here)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

# Resume training from the last checkpoint
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.022800,1.799960
2,0.658500,1.790109
3,0.765900,1.741502
4,0.881700,1.733232
5,0.631600,1.756891
6,0.523200,1.735805
7,0.687900,1.741971
8,0.567000,1.707045
9,0.527900,1.707781
10,0.519400,1.709331


TrainOutput(global_step=580, training_loss=0.6098134517669678, metrics={'train_runtime': 572.559, 'train_samples_per_second': 8.069, 'train_steps_per_second': 1.013, 'total_flos': 1217199282647040.0, 'train_loss': 0.6098134517669678, 'epoch': 10.0})

In [44]:
# تقييم النموذج
results = trainer.evaluate()
print("Evaluation results:", results)

Evaluation results: {'eval_loss': 1.7070454359054565, 'eval_runtime': 3.3494, 'eval_samples_per_second': 34.633, 'eval_steps_per_second': 4.478, 'epoch': 10.0}


In [45]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_tokenizer")

('./saved_tokenizer/tokenizer_config.json',
 './saved_tokenizer/special_tokens_map.json',
 './saved_tokenizer/vocab.txt',
 './saved_tokenizer/added_tokens.json',
 './saved_tokenizer/tokenizer.json')

In [46]:
import joblib
joblib.dump(label_encoder, "label_encoder.pkl")

['label_encoder.pkl']

In [47]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import joblib
import torch

#loading
model = AutoModelForSequenceClassification.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_tokenizer")
label_encoder = joblib.load("label_encoder.pkl")


In [62]:
data["Track"] = data["Track"].str.split(" / ")

In [64]:
expanded_data = data.explode("Track")

# print(expanded_data)

expanded_data["text"] = (
    "Track: " + expanded_data["Track"] + ". " +
    "Difficulty Level: " + expanded_data["Difficulty Level"] + ". " +
    "Description: " + expanded_data["course_description"]
)

print(expanded_data["text"].head())

0    Track: Data Analysis /Database Management Syst...
1    Track: Cybersecurity. Difficulty Level: Advanc...
2    Track: Cloud Computing. Difficulty Level: Adva...
2    Track: Backend Development. Difficulty Level: ...
3    Track: Cybersecurity. Difficulty Level: Interm...
Name: text, dtype: object


In [79]:
# تصفية الكورسات بناءً على Track و Difficulty Level
target_track = "Machine Learning"
target_level = "Intermediate"

filtered_courses = expanded_data[
    (expanded_data["Track"] == target_track) &
    (expanded_data["Difficulty Level"] == target_level)
]

# عرض الكورسات المصفاة
print(filtered_courses[["Course Name", "Track", "Difficulty Level"]])

                                           Course Name             Track  \
53   Predictive Modeling and Machine Learning with ...  Machine Learning   
61   Advanced Linear Models for Data Science 1: Lea...  Machine Learning   
115            (Non) Status quo Attitudes with NetLogo  Machine Learning   
126          Building AI Applications with Watson APIs  Machine Learning   
130                      Sample-based Learning Methods  Machine Learning   
132    Introduction to Trading, Machine Learning & GCP  Machine Learning   
138                     Machine Learning With Big Data  Machine Learning   
145                    Regression Modeling in Practice  Machine Learning   
152                 Machine Learning for Data Analysis  Machine Learning   
391  Microsoft Future Ready: Principles of Machine ...  Machine Learning   
394  An Introduction to Machine Learning in Quantit...  Machine Learning   
399  Critical Evaluation in Data Science: Data, the...  Machine Learning   
403  Microso

In [80]:
import torch

# قائمة لتخزين النتائج
course_scores = []

# التنبؤ بدرجات الكورسات المصفاة
for _, row in filtered_courses.iterrows():
    course_text = row["text"]  # النص الذي تم استخدامه للتدريب
    inputs = tokenizer(course_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    score = logits[0].max().item()  # أعلى درجة للفئة
    course_scores.append((row["Course Name"], score))

# ترتيب الكورسات حسب الدرجات
sorted_courses = sorted(course_scores, key=lambda x: x[1], reverse=True)


In [81]:
# عرض أعلى الكورسات
print(f"Top courses for Track: {target_track}, Difficulty Level: {target_level}:")
for i, (course_name, score) in enumerate(sorted_courses[:5]):  # أعلى 5 كورسات
    print(f"{i+1}. {course_name} (Score: {score:.2f})")

Top courses for Track: Machine Learning, Difficulty Level: Intermediate:
1. Critical Evaluation in Data Science: Data, the World, and You (Score: 7.71)
2. Microsoft Future Ready: Principles of Machine Learning with Python Programming (Score: 7.71)
3. An Introduction to Machine Learning in Quantitative Finance (Score: 7.71)
4. Data Science for Climate Change (Score: 7.71)
5. The Mathematics of Cryptography: From Ancient Rome to a Quantum Future (Score: 7.71)
